In [1]:
import pandas as pd
import pickle
import math

import numpy as np

from scipy.stats import spearmanr, kendalltau, SpearmanRConstantInputWarning
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning

In [2]:
tqdm.pandas()

Считаем данные:

In [3]:
players = None
with open('chgk/players.pkl', mode='rb') as f:
    players = pickle.load(f)
    
results = None
with open('chgk/results.pkl', mode='rb') as f:
    results = pickle.load(f)

tournaments = None
with open('chgk/tournaments.pkl', mode='rb') as f:
    tournaments = pickle.load(f)  

Выделим турниры для обучения и валидации:

In [4]:
def get_tournaments_by_year(year):
    return {tournament_id for tournament_id, tournament_data in tournaments.items() 
            if tournament_data['dateStart'][:4] == year}

In [5]:
tournaments_train = get_tournaments_by_year('2019')
tournaments_test = get_tournaments_by_year('2020')

Разделим данные турниров на train и test. При этом будем брать только те турниры, по которым определен результат (mask отлично от None). Также встречаются странные случаи, когда в одном турнире у различных команд длины mask (а следовательно количество вопросов) не совпадают. Я не понял, как такие случаи правильно интерпретировать, поэтому просто исключил их. Также оставил только те турниры, в которых mask состоит только из 0 и 1 (встречались случаи с X). Как вариант, можно было бы исключать такие вопросы, оставляя другие в данном турнире, однако для простоты такие туриниры исключил в целом. Вопросам присвоим сквозные идентификаторы. Данные по вопросам необходимо оставить тк в разных турнирах могли быть вопросы различной сложности и следовательно один и тот же игрок в различных турнирах мог показывать различную результативность (даже если не брать в расчет прочие факторы).

In [6]:
def split_train_test(tournament_results):
    train_data = []
    test_data = []
    correct_mask = {'0', '1'}

    # (tournament_id, team_id, player_id, question_id, result)

    last_question_id = 0

    for tournament_id, tournament_data in tournament_results.items():

        if len(tournament_data) == 0:
            continue

        if tournament_id in tournaments_train:
            data = train_data
        elif tournament_id in tournaments_test:
            data = test_data
        else:
            continue

        mask = tournament_data[0].get('mask', '')
        mask_len = len(mask) if mask is not None else 0

        current_data = []
        for team_data in tournament_data:
            team_id = team_data['team']['id']
            if team_data.get('mask', None) is None or not set(team_data['mask']).issubset(correct_mask) or len(team_data['mask']) != mask_len:
                current_data = []
                break
            for team_member_data in team_data['teamMembers']:
                player_id = team_member_data['player']['id']
                for i, result in enumerate(team_data['mask']):
                    question_id = last_question_id + i
                    current_data.append((tournament_id, team_id, player_id, question_id, int(result)))
        if current_data:
            data.extend(current_data)

        last_question_id += mask_len
    return train_data, test_data

In [7]:
train_data, test_data = split_train_test(results)

columns=['tournament_id', 'team_id', 'player_id', 'question_id', 'result']

train_data_df = pd.DataFrame(train_data, columns=columns)
test_data_df = pd.DataFrame(test_data, columns=columns)

Для того, чтобы постоить бейзлайн, можно воспользоваться следующими соображениями. Вероятность правильно ответить на вопрос тем больше, чем сильнее игрок и чем проще вопрос. Тогда мы можем обучить логистическую регрессию, у которой в X_train будет в каждой строке игрок и вопрос, а целевой переменной будет 0 или 1 - ответил ли игрок на вопрос верно (причем пока будем считать, что если команда ответила на вопрос верно, то и игрок ответил верно). Тогда получим:

$$
\sigma(w_1 \cdot player_i + w_2 \cdot question_j) = p_{ij}
$$
где $p_{ij}$ - вероятность того, что игрок $player_i$ ответит правильно на вопрос $question_j$

При этом вес $w_1$ при игроке будет характеризовать насколько игрок силен, а вес при вопросе $w_2$ - насколько вопрос прост.

In [8]:
ohe = OneHotEncoder(handle_unknown='ignore')

X_train = ohe.fit_transform(train_data_df[['player_id', 'question_id']])
y_train = train_data_df['result']

In [9]:
simplefilter("ignore", category=ConvergenceWarning)

baseline_model = LogisticRegression(random_state=42)
baseline_model.fit(X_train, y_train)

LogisticRegression(random_state=42)

В тесте у нас идентификаторы вопросов не пересекаются с train. Следовательно при one-hot кодировании признаки в тесте станут нулевыми. Таким образом в тесте вопросы не будут влиять на результат. То есть мы будет учитывать только состав команд.

In [10]:
X_test = ohe.transform(test_data_df[['player_id', 'question_id']])
predicted = baseline_model.predict_proba(X_test)

Для подсчета рейтинга команды будем считать долю правильно отвеченных вопросов. Т.е., например, если команда ответила правильно на 8 из 10 вопросов, то рейтинг будет равен 0.8. Очевидным образом это считается для известных ответов. Чтобы посчитать предсказание рейтинга команды можно пойти 2 путями. Во-первых, если у нас есть верооятность игрока ответить на вопрос, то мы можем для кажого вопроса просто провести серию испытаний бернулли с данной вероятностью для каждого игрока. И тогда либо взять максимум по вопросу (это равнозначно тому, что если хотя бы один игрок ответил на вопрос, то команда тоже ответила на вопрос). Но мне показалось более естественным взять не максимум, а среднее. Это отражает интуитивное представление, что если в команде один человек дал правильный ответ, то это команда будет более слабой чем та, в которой правильный ответ дали 3 человека. Во-вторых, можно не проводить серию испытаний бернулли, а напрямую брать (усреднять) вероятность (или величину характеризующую вероятность, что потребуется в дальнейшем) игроков ответить на вопрос. Второй подход показывает лучшие результаты и обобщается на дальнейшие вычисления, поэтому будем использовать его.

In [11]:
def calc_team_ratings(team_data, test_rating_data):
    team_ratings = team_data[['question_id', 'result', 'predict']].groupby(by='question_id').mean().mean()
    first_row = team_data.iloc[0]
    tournament_id = int(first_row.tournament_id)
    team_id = int(first_row.team_id)
    test_rating_data.append([tournament_id, team_id, team_ratings.result, team_ratings.predict]) 

In [12]:
def calc_metrics(df_input, predicted):
    df = df_input.copy()
    df['predict'] = predicted
        
    test_rating_data = []
    df.groupby(by=['tournament_id', 'team_id']).progress_apply(lambda d: calc_team_ratings(d, test_rating_data))
    
    test_rating_df = pd.DataFrame(test_rating_data, columns=['tournament_id', 'team_id', 'rating_true', 'rating_predicted'])
    
    spearmanr_results = []
    kendalltau_results = []
    for tournament_id in test_rating_df['tournament_id'].unique():
        cur = test_rating_df[test_rating_df['tournament_id'] == tournament_id]
        spearmanr_correlation = spearmanr(cur['rating_true'], cur['rating_predicted']).correlation
        kendalltau_correlation = kendalltau(cur['rating_true'], cur['rating_predicted']).correlation
        if not np.isnan(spearmanr_correlation):
            spearmanr_results.append(spearmanr_correlation)
        if not np.isnan(kendalltau_correlation):
            kendalltau_results.append(kendalltau_correlation)
    
    return {
        'spearmanr': np.mean(np.array(spearmanr_results)),
        'kendalltau': np.mean(np.array(kendalltau_results))
    }

Метрики на train:

In [13]:
simplefilter("ignore", category=SpearmanRConstantInputWarning)

train_predicted = baseline_model.predict_proba(X_train)
calc_metrics(train_data_df, train_predicted[:, 1])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65154/65154 [01:50<00:00, 591.03it/s]


{'spearmanr': 0.8423995028268146, 'kendalltau': 0.6952657410177617}

Метрики на test:

In [14]:
calc_metrics(test_data_df, predicted[:, 1])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16992/16992 [00:28<00:00, 601.69it/s]


{'spearmanr': 0.7547225234413555, 'kendalltau': 0.6014348428024349}

Теперь попробуем усовершенствовать наш подход. Вместо того, чтобы считать, что если команда ответила правильно, то и игрок ответил правильно (как это было в бейзлайне), попробуем посчитать условную вероятность $P(player_i \; answers \; question | team \; answers \; question)$

$$
P(player_i \; answers \; question | team \; answers \; question) = \dfrac{P(team \; answers \; question | player_i \; answers \; question) \cdot P(player_i \; answers \; question)} {P(team \; answers \; question)}
$$

Как и прежде будем считать, что если игрок ответил на вопрос, то и команда ответила на вопрос. Тогда первый множитель в числителе будет равен 1. Второй множитель мы вычислили раньше и данее будем пересчитывать. Вероятность того, что команда ответила на вопрос, будем считать как вероятность того, что хотя бы один игрок из состава ответил на вопрос. 

В целом, предположение, что если игрок ответил на вопрос верно, то и команда ответила на вопрос верно, все равно кажется слишком сильным, потому что команда могла выбрать версию другого игрока. Но более сложные подходы, к сожалению, не получились.

Тогда Е-шаг будет заключаться в расчете указанных условных вероятностей верного ответа игрока при уловии, что команда ответила верно. В тоом случае, когда команда ответила неверно, будем считать, что никто из игроков не ответил верно (опять таки довольно сильное предположение). Далее, эти вероятности мы возьмем в качестве целевых переменных и на M-шаге переобучим нашу модель на новый таргет, откуда получим новые априорные вероятности для игроков и тд в цикле.

Здесь возникает новая сложность. Когда мы посчитаем новый таргет, то это уже будут различные вещественные числа, а не только 0 и 1. Следовательно, логистическая регрессия из sklearn уже работать не будет. С этим можно побороться разными способами, но у меня лучше показал себя следующий вариант: вместо логистической регрессии на M-шаге будем обучать линейную регрессию, т.е. будем стараться спрогнозировать данную условную вероятность. Тут как раз пригодится то, что при расчете метрики мы не стали завязываться на то, что моогут быть в значениях только 0 и 1. Но для Е-шага надо будет получить именно вероятности, а линейная регрессия может предсказать и что-то вне допустимого диапазона. Тогда чтобы получить данные для Е-шага дополнительно еще возьмем от результата сигмоиду. Я не до конца уверен в корректности этого, но, как будет видно ниже, этот подход работает.

In [15]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [16]:
def e_step(df, predict_proba):
    
    df['predict_proba'] = predict_proba
    
    new_scores_store = {}
    
    def calc_new_scores(d):
        first_row = d.iloc[0]
        new_scores_store[(first_row.tournament_id, first_row.team_id)] = 1 - np.prod(1 - d['predict_proba'])
        
    
    df[train_data_df['result'] == 1].groupby(by=['tournament_id', 'team_id']).apply(calc_new_scores)
    df['new_target'] = df.progress_apply(lambda r: r.predict_proba / new_scores_store.get((r.tournament_id, r.team_id), 1), axis=1)
    df.loc[df['result'] == 0, 'new_target'] = 0    

In [17]:
def m_step(train_data_df, X_train, X_test):
    model = Ridge(random_state=42)
    model.fit(X_train, train_data_df['new_target'])
    
    train_predicted = model.predict(X_train)
    print('train metrics')
    print(calc_metrics(train_data_df, train_predicted))
    
    predicted = model.predict(X_test)
    print('test metrics')
    print(calc_metrics(test_data_df, predicted))
    
    return np.array(list(map(sigmoid, train_predicted)))

In [18]:
predict_proba = train_predicted[:, 1]
for step in range(4):
    print(f'step {step + 1}')
    e_step(train_data_df, predict_proba)
    predict_proba = m_step(train_data_df, X_train, X_test)

step 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13454728/13454728 [04:56<00:00, 45332.32it/s]


train metrics


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65154/65154 [02:01<00:00, 538.40it/s]


{'spearmanr': 0.8492243482658097, 'kendalltau': 0.7031497476968471}
test metrics


...

train metrics


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65154/65154 [01:49<00:00, 594.78it/s]


{'spearmanr': 0.856496508488362, 'kendalltau': 0.7149774586629053}
test metrics


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16992/16992 [00:28<00:00, 601.72it/s]


{'spearmanr': 0.7620848121123741, 'kendalltau': 0.6098620531692226}


Видим, что результаты понемногу, но улучшаются.


Теперь построим рейтинг турниров по сложности вопросов. Для того, чтобы получить рейтинг турнира по сложности, просуммируем сложности всех вопросов, входящих в турнир. Сложности вопросов возьмем из бейзлайна (коэффициент при вопросе), точнее там стоит величина противоположная сложности, так что учтем это при сортировке. Здесь взят бейзлайн, потому что в силу постоения EM-алгоритма, там эта информация уже потеряна. Однако, т.к качество относительно бейзлайна выросло не очень сильно, то в целом это не должно заметно повлиять на результаты.

In [19]:
total_questions = ohe.categories_[1].shape[0]

questions_inv_complexity = {}
for question_id, inv_complexity in zip(ohe.categories_[1], baseline_model.coef_[0][-total_questions:]):
    questions_inv_complexity[question_id] = inv_complexity

In [20]:
train_data_df['tournament_name'] = train_data_df['tournament_id'].apply(lambda key: tournaments[key]['name'])
train_data_df['question_inv_complexity'] = train_data_df['question_id'].apply(lambda key: questions_inv_complexity[key])

tournament_and_inv_complexity = train_data_df[['tournament_id', 'tournament_name', 'question_inv_complexity']].groupby(by=['tournament_id', 'tournament_name']).sum()
tournament_ordered_by_questions = tournament_and_inv_complexity.sort_values(by='question_inv_complexity', ascending=True)

Посмотрим на топ самых сложных турниров:

In [21]:
tournament_ordered_by_questions[:15]

,,question_inv_complexity
tournament_id,tournament_name,
5025,Кубок городов,-413070.180968
6149,Чемпионат Санкт-Петербурга. Первая лига,-227101.480729
5516,Синхрон Моносова,-130977.749256
5098,"Ра-II: синхрон ""Борского корабела""",-102580.771788
5465,Чемпионат России,-72856.930710
5159,Первенство правого полушария,-67397.086440
5074,Синхрон Моносова,-67002.630501
5741,All Cats Are Beautiful,-59614.913691
5188,Турнирум,-52081.216815


И на топ снизу:

In [22]:
tournament_ordered_by_questions[-15:]

,,question_inv_complexity
tournament_id,tournament_name,
5954,Школьная лига. II тур.,101844.236936
5011,(а)Синхрон-lite. Лига старта. Эпизод IV,103743.801717
5702,(а)Синхрон-lite. Лига старта. Эпизод IX,105853.598338
5855,Лига вузов. IV тур,106062.377963
5853,Лига вузов. II тур,109118.112255
5698,(а)Синхрон-lite. Лига старта. Эпизод VII,109792.750373
5955,Школьная лига. III тур.,109970.871778
5854,Лига вузов. III тур,110891.490685
5820,ОВСЧ. 3 этап,119205.492017


В целом похоже на правду: вверху, судя по названиям, более сложные турниры.

Теперь попробуем обучить бейзлайн модель не только на 2019 годе, но и на всех предыдущих.

In [28]:
min_year = int(min([v['dateEnd'][:4] for k, v in tournaments.items()]))

In [34]:
tournaments_train = set()
for year in range(min_year, 2020):
    tournaments_train = tournaments_train.union(get_tournaments_by_year(str(year)))

In [36]:
train_data, _ = split_train_test(results)
train_data_df = pd.DataFrame(train_data, columns=['tournament_id', 'team_id', 'player_id', 'question_id', 'result'])

Также добавим в датасет колонку с годом.

In [44]:
train_data_df['year'] = train_data_df['tournament_id'].progress_apply(lambda t_id: int(tournaments[t_id]['dateEnd'][:4]))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63548684/63548684 [01:18<00:00, 812486.86it/s]


In [45]:
train_data_df

,tournament_id,team_id,player_id,question_id,result,year
0,22,1,1560,0,0,2004
1,22,1,1560,1,1,2004
2,22,1,1560,2,1,2004
3,22,1,1560,3,1,2004
4,22,1,1560,4,0,2004
...,...,...,...,...,...,...
63548679,6349,39267,63355,194754,0,2011
63548680,6349,39267,63355,194755,0,2011
63548681,6349,39267,63355,194756,0,2011
63548682,6349,39267,63355,194757,0,2011


In [37]:
ohe = OneHotEncoder(handle_unknown='ignore')

X_train = ohe.fit_transform(train_data_df[['player_id', 'question_id']])
y_train = train_data_df['result']

In [39]:
baseline_model = LogisticRegression(random_state=42)
baseline_model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [40]:
X_test = ohe.transform(test_data_df[['player_id', 'question_id']])
predicted = baseline_model.predict_proba(X_test)

In [41]:
simplefilter("ignore", category=SpearmanRConstantInputWarning)

train_predicted = baseline_model.predict_proba(X_train)
calc_metrics(train_data_df, train_predicted[:, 1])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 279698/279698 [09:46<00:00, 477.18it/s]


{'spearmanr': 0.8072826983389103, 'kendalltau': 0.6535829656892439}

In [42]:
calc_metrics(test_data_df, predicted[:, 1])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16992/16992 [00:31<00:00, 540.80it/s]


{'spearmanr': 0.7171060627801068, 'kendalltau': 0.5631238148342792}

Как видим, качество стало хуже (как на train, так и на test). Видимо, это связано с тем, что данных стало больше, и поэтому надо бы увеличить параметр max_iter, но, боюсь, ноутбук этого уже может не осилить. Кроме того, не очень понятно, насколько справедливо тогда будет сравнивать метрики со старым бейзлайном.

Тем не менее, несмотря на то, что улучшения метрик на этом этапе добиться не удалось, но попробуем побороться с эффектом того, что первые неудачные турниры участника будут тянуть его рейтинг вниз всю жизнь. Для этого, вместо того, чтобы просто рассматривать только, например, последний год, попробуем брать всю историю, но более старым годам давать меньший вес. Для этого в принципе подойдет любая монотонно возрастающая функция. Возьмем в качестве примера $f(x) = x^2$, т.е. почтитаем веса следующим образом:

In [53]:
min_presented_year = train_data_df['year'].values.min()
weights = (train_data_df['year'].values - min_presented_year + 1) ** 2

И теперь заново обучим модель:

In [54]:
baseline_model = LogisticRegression(random_state=42)
baseline_model.fit(X_train, y_train, sample_weight=weights)

LogisticRegression(random_state=42)

In [55]:
X_test = ohe.transform(test_data_df[['player_id', 'question_id']])
predicted = baseline_model.predict_proba(X_test)

In [56]:
simplefilter("ignore", category=SpearmanRConstantInputWarning)

train_predicted = baseline_model.predict_proba(X_train)
calc_metrics(train_data_df, train_predicted[:, 1])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 279698/279698 [08:51<00:00, 526.63it/s]


{'spearmanr': 0.8081028859478474, 'kendalltau': 0.6543193982221512}

In [57]:
calc_metrics(test_data_df, predicted[:, 1])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16992/16992 [00:30<00:00, 562.20it/s]


{'spearmanr': 0.7299030714183263, 'kendalltau': 0.5762940760837115}

Видим, что метрики действительно улучшились! Т.е. такой подход в целом имеет смысл.